# 単回帰分析を使って、FXのモデル仮説をする

## ０．ライブラリをインポートする

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## １．ヒストリカルデータを読み込む

In [24]:
dfSecond = pd.read_csv('data/EURJPY_2017_all.csv')

In [25]:
dfSecond.head(3)

,Date,Time,Open,High,Low,Close,Volume
0,2017.01.03,00:00,122.719,122.749,122.656,122.670,19.0
1,2017.01.03,00:01,122.650,122.720,122.626,122.626,27.0
2,2017.01.03,00:02,122.676,122.681,122.637,122.681,9.0


In [28]:
# 行数をカウント
len(dfSecond)

371280

In [26]:
# データの行数と列数を表示する
dfSecond.shape

(371280, 7)

In [27]:
# 各値の統計値は？
dfSecond.describe()

,Open,High,Low,Close,Volume
count,371279.000000,371279.00000,371279.000000,371279.000000,371279.000000
mean,126.655996,126.66719,126.644853,126.656010,125.550656
std,5.432699,5.43218,5.433242,5.432722,74.440637
min,114.857000,114.87100,114.845000,114.855000,1.000000
25%,121.802000,121.81500,121.788000,121.802000,72.000000
50%,128.166000,128.17900,128.155000,128.166000,113.000000
75%,132.063000,132.07200,132.053000,132.063000,165.000000
max,135.507000,135.51000,135.482000,135.507000,582.000000


## ２．コードを書いて、一円のアップ／ダウンを抽出する

やること<br>
37万件強の為替データ(dfSecond)の中から、動きがあった場所を抽出する。<br>
一円のアップ／ダウンがあった行だけのデータフレーム(dfMoving)を作る

1. Open列をDecimaiにしたDecOpen列を追加する
2. DecOpenの値を一行前と比較する
3. Movement列を追加し、プラスの変化があれば"up"、マイナスの変化があれば"down"を設定する
4. upまたはdownの行だけを抽出する

暫定対応：ヒストリカルデータの中から、毎6時間ごとのデータを取り出す

### 2.1.Open列をDecimalにしたDecOpen列を追加する

暫定対応：ヒストリカルデータの中から毎6時間毎のデータを取り出し、別のDFにコピーする

In [29]:
# これは、試しにTime列を：で分割したもの
dfSecond['Hur'] = dfSecond['Time'].str.split(':')

In [30]:
# dfHourに新しい列を追加するため、dfをコピーする必要がある。
dfHour = dfSecond[dfSecond['Time'].isin(['00:00', '06:00', '12:00', '18:00'])].copy()
dfHour

,Date,Time,Open,High,Low,Close,Volume,Hur
0,2017.01.03,00:00,122.719,122.749,122.656,122.670,19.0,"[00, 00]"
360,2017.01.03,06:00,123.065,123.072,123.063,123.066,89.0,"[06, 00]"
720,2017.01.03,12:00,122.894,122.898,122.857,122.874,335.0,"[12, 00]"
1080,2017.01.03,18:00,122.562,122.580,122.543,122.564,400.0,"[18, 00]"
1440,2017.01.04,00:00,122.479,122.548,122.416,122.541,17.0,"[00, 00]"
1798,2017.01.04,06:00,122.708,122.730,122.705,122.705,145.0,"[06, 00]"
2158,2017.01.04,12:00,122.835,122.892,122.834,122.866,245.0,"[12, 00]"
2518,2017.01.04,18:00,122.768,122.854,122.749,122.843,318.0,"[18, 00]"
2878,2017.01.05,00:00,122.895,122.918,122.895,122.901,8.0,"[00, 00]"
3237,2017.01.05,06:00,122.536,122.554,122.529,122.553,134.0,"[06, 00]"


In [31]:
# numpyのtrunc関数を使って、Open列を整数に切り捨てる
dfHour['DecOpen'] = np.trunc(dfHour['Open'])

### 2.2.DecOpenの値を一行前と比較する

In [32]:
dfHour['diff'] = (dfHour['DecOpen'] - dfHour['DecOpen'].shift(1)).fillna(0)

### 2.3.Movement列を追加し、プラスの変化があれば"up"、マイナスの変化があれば"down"を設定する

In [123]:
dfHour.loc[dfHour['diff'] < 0, 'Movement'] = 'down'
dfHour.loc[dfHour['diff'] > 0, 'Movement'] = 'up'

### 2.4.upまたはdownの行だけを抽出する

In [129]:
dfMovements = dfHour[dfHour['Movement'].isin(['up', 'down'])].copy()

In [130]:
dfMovements.describe()

,Open,High,Low,Close,Volume,DecOpen,diff
count,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000
mean,126.142786,126.161448,126.123060,126.142004,175.044355,125.653226,0.052419
std,5.378069,5.377564,5.378411,5.380393,128.302882,5.414846,1.030543
min,114.951000,114.970000,114.951000,114.965000,2.000000,114.000000,-1.000000
25%,121.193250,121.203500,121.172000,121.191750,48.250000,121.000000,-1.000000
50%,125.038500,125.043000,125.037500,125.038000,169.500000,125.000000,1.000000
75%,131.358750,131.370250,131.315750,131.360000,278.250000,131.000000,1.000000
max,135.273000,135.304000,135.257000,135.283000,482.000000,135.000000,4.000000


## ３．一円の上下があった場所の法則性を見る